# <center> Million Songs Analysis</center>

## Starting the Spark session

In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("local[1]") \
        .appName("Project_19")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()
            
spark_context = spark_session.sparkContext

## Importations

In [2]:
import h5py
import io, time
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id, col, when

## Loading the data into RDDs (Skip)

In [36]:
""" This function turns a list of binary elements to a string with the element separated by the character ',' """
def list_bin_to_str(list_terms) :
    string=""
    for term in list_terms :
        string += term.decode()+','
    return string[1:-1]   

In [37]:
""" We prepare first a RDD containing tuples (song_information, artist_information) """
#rdd = spark_context.binaryFiles("hdfs://host-192-168-1-153-ldsa:9000/millionsongs/data/A/B/*")
rdd = spark_context.binaryFiles("/mnt/ms/data/A/B/*")

# A function to open the h5 files one by one and take the information of interest for the song and the artist
def f(x):
    with h5py.File(io.BytesIO(x[1])) as f:
        
        f_meta = f['metadata']["songs"]
        f_analys = f['analysis']["songs"]
        f_brainz = f['musicbrainz']["songs"]
        
        artist_terms = list_bin_to_str(f['metadata']['artist_terms'][:])
        similar_artists = list_bin_to_str(f['metadata']['similar_artists'][:])
        
        return ((f_meta[0][-3].decode(), f_meta[0][-2].decode(),
                f_meta[0][9].decode(), f_meta[0][4].decode(),
                f_meta[0][-6].decode(),
                float(f_analys[0][3]), float(f_meta[0][-4]),
                float(f_analys[0][2]), float(f_analys[0][23]), 
                float(f_analys[0][-4]), int(f_brainz[0][1])), 
               
                (f_meta[0][4].decode(), f_meta[0][9].decode(),
                f_meta[0][6].decode(), float(f_meta[0][3]),
                artist_terms, similar_artists))

rdd = rdd.map(f)

""" We create two RDDs (songs, artists) from the general one """
rdd_songs = rdd.map(lambda x : x[0])
rdd_artists = rdd.map(lambda x : x[1])

## Turning RDDs to Data frames (Skip)

<b> *   Table Songs </b>

In [7]:
attributes = rdd_songs.map(lambda p: Row(p[0], p[1], p[2], p[3], p[4], p[5], p[6], p[7], p[8], p[9], p[10]))
fields = [StructField("song_id", StringType(), True), StructField("title", StringType(), True), 
          StructField("artist_name", StringType(), True), StructField("artist_id", StringType(), True),
          StructField("release_album", StringType(), True), 
          StructField("duration", FloatType(), True), StructField("hotness", FloatType(), True), 
          StructField("danceability", FloatType(), True), StructField("loudness", FloatType(), True), 
          StructField("tempo", FloatType(), True), StructField("year", IntegerType())]
songs_schema = StructType(fields)

df_songs = spark_session.createDataFrame(attributes, songs_schema)
df_songs.show()

+------------------+--------------------+--------------------+------------------+--------------------+---------+----------+------------+--------+-------+----+
|           song_id|               title|         artist_name|         artist_id|       release_album| duration|   hotness|danceability|loudness|  tempo|year|
+------------------+--------------------+--------------------+------------------+--------------------+---------+----------+------------+--------+-------+----+
|SOPLQMB12AC4686313|Girl Of Mysteriou...|           MARC COHN|ARWRVSR1187FB575FB|    Burning The Daze| 257.5669|       0.0|         0.0| -13.107|102.073|1998|
|SOFJWKK12AB01826CD|The Cookie Bakers...|      Laurie Berkner|ARC1CJB1187B99FCA4|           Buzz Buzz|233.45587|       0.0|         0.0| -16.263| 144.71|2001|
|SOHNDZB12A8C13EDB8|      Shooting Stars|           Cauterize|ARF0COO1187B99B9EF|    So Far From Real|240.43057|0.71432906|         0.0|  -4.714| 97.148|2003|
|SOAPGDK12A6D4FAF5E|        The Blessing|     

<b> *   Table Artists </b>

In [38]:
attributes = rdd_artists.map(lambda p: Row(p[0], p[1], p[2], p[3], p[4], p[5]))
fields = [StructField("id", StringType(), True), StructField("name", StringType(), True), 
          StructField("location", StringType(), True), StructField("hotness", FloatType(), True),
          StructField("terms", StringType(), True), 
          StructField("similar_artists", StringType(), True)]
artists_schema = StructType(fields)

df_artists = spark_session.createDataFrame(attributes, artists_schema)\
                          .dropDuplicates()
df_artists.show()

+------------------+--------------------+------------------+----------+--------------------+--------------------+
|                id|                name|          location|   hotness|               terms|     similar_artists|
+------------------+--------------------+------------------+----------+--------------------+--------------------+
|ARPET7C1187B996C8B|          Bajja Jedd|                  |       0.0|ancehall,reggae,j...|RCQ6GG1187B9A39DB...|
|ARTBBJ51187FB498F5|      Barrett Strong|     Westpoint, MS|0.38878328|otown,jazz funk,l...|RV03D41187FB413E8...|
|ARH39ZK1187B9ABC22|           Girl Talk|   California - SF| 0.5783318|litch,smooth jazz...|RFL5HZ1187B9AC31B...|
|ARYEUNC1187B9A3648|     utopia:banished|                  |0.33196872|roken beat,grunge...|R7TT521187B99DF76...|
|AR6AYVY1187B990E7C|          Tony Rebel|                  | 0.4064398|ancehall,roots re...|RVJGJE1187FB436EB...|
|ARX8MDO1187FB5A1B9| Metabass 'N' Breath|                  |       0.0|ip hop,breakbeat,

## Saving data frames in csv files (Skip)

<b> *   Table Songs </b>

In [8]:
df_songs.write.format('com.databricks.spark.csv')\
        .option("header", "true")\
        .save('/home/ubuntu/MySongs.csv')


<b> *   Table Artists </b>

In [40]:
df_artists.write.format('com.databricks.spark.csv')\
        .option("header", "true")\
        .save('/home/ubuntu/MyArtists.csv')


## Loading data frames from csv files

<b> *   Table Songs </b>

In [3]:
fields = [StructField("song_id", StringType(), True), StructField("title", StringType(), True), 
          StructField("artist_name", StringType(), True), StructField("artist_id", StringType(), True),
          StructField("release_album", StringType(), True), 
          StructField("duration", FloatType(), True), StructField("hotness", FloatType(), True), 
          StructField("danceability", FloatType(), True), StructField("loudness", FloatType(), True), 
          StructField("tempo", FloatType(), True), StructField("year", IntegerType())]
songs_schema = StructType(fields)

df_songs = spark_session.read.load('file:/home/ubuntu/MySongs.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          schema=songs_schema)\
                        .cache()


<b> *   Table Artists </b>

In [4]:
fields = [StructField("id", StringType(), True), StructField("name", StringType(), True), 
          StructField("location", StringType(), True), StructField("hotness", FloatType(), True),
          StructField("terms", StringType(), True), 
          StructField("similar_artists", StringType(), True)]
artists_schema = StructType(fields)

df_artists = spark_session.read.load('file:/home/ubuntu/MyArtists.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          schema=artists_schema)\
                          .cache()

# Functions

In [5]:
""" This function asks the user for the title of a song and returns its attributes in the database if it exists """
def ask_for_song() :
    song_title = input(" Title of the song : ")
    song = df_songs.filter(df_songs["title"].like("%" + song_title + "%"))
    while song.count() == 0 :
        print(" This song doesn't exist in the database")
        song = df_songs.filter(df_songs["title"].like("%" + input("Title of the song : ") + "%"))
    print(" The song is : ", song.first()["title"])
    return song.first()

In [6]:
""" This function returns the songs which belong to the same artist as the song input (Row) """
def same_artist(song) :
    similar_songs = df_songs.filter(df_songs.artist_name == song["artist_name"])
    if similar_songs.count() > 1 : 
        return similar_songs.filter(similar_songs.title != song["title"])
    else :
        return None

In [26]:
def similar_tempo(song, margin) : 
    similar_songs = df_songs.filter(df_songs.tempo.between(song["tempo"] - margin, song["tempo"] + margin))\
                      .filter(df_songs.title != song["title"])
    if similar_songs.count() == 0 :
        similar_songs = None
    return similar_songs.collect()

In [15]:
def similar_loudness(song, margin) : 
    similar_songs = df_songs.filter(df_songs.loudness.between(song["loudness"] - margin, song["loudness"] + margin))\
                      .filter(df_songs.title != song["title"])
    if similar_songs.count() == 0 :
        similar_songs = None
    return similar_songs.collect()

## Core Analysis

In [11]:
user_song = ask_for_song()

 Title of the song : Rattlesnake!
 The song is :  Rattlesnake!


In [21]:
songs_same_artist = same_artist(user_song)
if songs_same_artist != None :
    some_songs = songs_same_artist.take(5)
    print(" Songs from the same artist :", end ='\t')
    for song in some_songs : 
        print(song["title"], sep = ", ")
    

 Songs from the same artist :	God Bless You (Goddamn it)


In [39]:
songs_similar_tempo = similar_tempo(user_song, 1)
if songs_similar_tempo != None :
    print(" Songs with similar tempo :", end ='\t')
    for song in songs_similar_tempo : 
        print(song["title"], end = ", ")

 Songs with similar tempo :	Fast Girls, High Tonight, Ma Promese, Road Song, My Heart is Yours, Chlebnikov, 

In [42]:
songs_similar_loudness = similar_loudness(user_song, 0.1)
if songs_similar_loudness != None :
    print(" Songs with similar loudness :", end ='\t')
    for song in songs_similar_loudness : 
        print(song["title"], end = ", ")

 Songs with similar loudness :	Guajira, I'll Be Autumn_ You Be Winter, Je Me Lâche, Something, Sunshine After The Rain, The Lair Of The White Worm, Heaven, Kinda New (We All Live & Die), She's Gone, Psychobilly Is All Around, The Nothing Doctrine, Medley Leïla Chico, Other Girls, Kids In America, Stanotte, Self Destruct, Ringa Ding Ding, 

## Tests (To delete) 

In [4]:
songs = h5py.File('b.h5') # type = HDF5 file.

print(f'keys are {list(songs.values())}'
group_analysis = songs['metadata'] # type: HDF5 group.
print()
print(group_analysis.keys()) # AttributeManager -- these are like a small metadata dictionary attached anywhere in the tree.

#bars_start = songs['analysis']['bars_start'] # Dataset.

# Lets look at a more complex table elsewhere in the tree...

songs = songs['metadata']['songs'] # Dataset.

# This dataset has rows and columns. Lets get the first row...
song_row = songs.value
print()
# print the column names
print(song_row.dtype.names)
print(song_row)
print(song_row.dtype.names[3])
print(song_row[0][3])
#print([song.decode("ASCII") for song in song_row])


keys are [<HDF5 group "/analysis" (16 members)>, <HDF5 group "/metadata" (5 members)>, <HDF5 group "/musicbrainz" (3 members)>]

<KeysViewHDF5 ['artist_terms', 'artist_terms_freq', 'artist_terms_weight', 'similar_artists', 'songs']>

('analyzer_version', 'artist_7digitalid', 'artist_familiarity', 'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_mbid', 'artist_name', 'artist_playmeid', 'genre', 'idx_artist_terms', 'idx_similar_artists', 'release', 'release_7digitalid', 'song_hotttnesss', 'song_id', 'title', 'track_7digitalid')
[(b'', 165270, 0.58179377, 0.40199754, b'ARD7TVE1187B99BFB1', nan, b'California - LA', nan, b'e77e51a5-4761-45b3-9847-2051f811e366', b'Casual', 4479, b'', 0, 0, b'Fear Itself', 300848, 0.60211999, b'SOMZWCG12A8C13C480', b"I Didn't Mean To", 3401791)]
artist_hotttnesss
0.4019975433642836


/home/ubuntu/.local/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


## Releasing the resources

In [106]:
spark_context.stop()